In [5]:
import sys, os
desiredPath = "C:/Users/RAMA/CoppeliaScripts"
os.chdir(desiredPath)
try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')

import time 
import sys
import numpy as np

if os.path.isfile('/'.join([desiredPath, 'vrepConst.py'])):
    import vrepConst as wlvConst
else:
    import simConst as wlvConst

In [6]:
class Dr20Robot_Interface:
    def __init__(self, sim, clientID):
        self._sim = sim
        self.clientID = clientID
        err, self.robHandle = sim.simxGetObjectHandle(clientID, "dr20", sim.simx_opmode_oneshot_wait)
        if err > 0:
            print("*** Error getting robot handle {}".format(err))
        
        errCode = [0] * 7
        errCode[0], self.leftMotor = sim.simxGetObjectHandle(clientID, "dr20_leftWheelJoint_", sim.simx_opmode_oneshot_wait)
        errCode[1], self.rightMotor = sim.simxGetObjectHandle(clientID, "dr20_rightWheelJoint_", sim.simx_opmode_oneshot_wait)
        errCode[2], self.infrared1 = sim.simxGetObjectHandle(clientID, "dr20_infraredSensor1_", sim.simx_opmode_oneshot_wait)
        errCode[3], self.infrared2 = sim.simxGetObjectHandle(clientID, "dr20_infraredSensor2_", sim.simx_opmode_oneshot_wait)
        errCode[4], self.infrared5 = sim.simxGetObjectHandle(clientID, "dr20_infraredSensor5_", sim.simx_opmode_oneshot_wait)
        errCode[5], self.infrared6 = sim.simxGetObjectHandle(clientID, "dr20_infraredSensor6_", sim.simx_opmode_oneshot_wait)
        errCode[6], self.ultasonic = sim.simxGetObjectHandle(clientID, "dr20_ultrasonicSensor_", sim.simx_opmode_oneshot_wait)
        if sum(errCode) > 0:
            print("*** Error initialising robot {}".format(errCode))
        
        for _ in range(10):
            sim.simxReadProximitySensor(clientID, self.infrared1, sim.simx_opmode_streaming)
            sim.simxReadProximitySensor(clientID, self.infrared2, sim.simx_opmode_streaming)
            sim.simxReadProximitySensor(clientID, self.infrared5, sim.simx_opmode_streaming)
            sim.simxReadProximitySensor(clientID, self.infrared6, sim.simx_opmode_streaming)
            sim.simxReadProximitySensor(clientID, self.ultasonic, sim.simx_opmode_streaming)
        
        self.v0 = 0.4
        self.wheelDiameter = 0.085
        self.interWheelDistance = 0.254
        self.noDetectionDistance = 0.4  # Adjusted to match the Lua script
        self.viewSensors = False
        self.brait_left = [0, -0.5, -1.25, -1, -0.2]

    def rotate_right(self, speed=2.0):
        self._set_two_motor(speed, -speed)

    def rotate_left(self, speed=2.0):
        self._set_two_motor(-speed, speed)

    def move_forward(self, speed=2.0):
        self._set_two_motor(speed, speed)

    def move_backward(self, speed=2.0):
        self._set_two_motor(-speed, -speed)

    def _set_two_motor(self, left: float, right: float):
        self._sim.simxSetJointTargetVelocity(self.clientID, self.leftMotor, left, self._sim.simx_opmode_blocking)
        self._sim.simxSetJointTargetVelocity(self.clientID, self.rightMotor, right, self._sim.simx_opmode_blocking)

    def stop(self):
        self._set_two_motor(0.0, 0.0)

    def _readSensor(self, sensorHandle, sensorType):
        if sensorType == sim.sim_object_proximitysensor_type:
            err_code, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = self._sim.simxReadProximitySensor(self.clientID, sensorHandle, self._sim.simx_opmode_buffer)
            distance = np.linalg.norm(detectedPoint)
            if err_code == 0 and detectionState:
                return sim.simx_return_ok, distance
            return 1, self.noDetectionDistance
        else:
            raise Exception('Unknown sensor type {} in _readSensor'.format(sensorType))

    def readSensors(self):
        sensReading = [self.noDetectionDistance] * 5
        err, dist = self._readSensor(self.infrared5, sim.sim_object_proximitysensor_type)
        sensReading[0] = dist if err == sim.simx_return_ok and dist < self.noDetectionDistance else self.noDetectionDistance
        err, dist = self._readSensor(self.infrared6, sim.sim_object_proximitysensor_type)
        sensReading[1] = dist if err == sim.simx_return_ok and dist < self.noDetectionDistance else self.noDetectionDistance
        err, dist = self._readSensor(self.ultasonic, sim.sim_object_proximitysensor_type)
        sensReading[2] = dist if err == sim.simx_return_ok and dist < self.noDetectionDistance else self.noDetectionDistance
        err, dist = self._readSensor(self.infrared1, sim.sim_object_proximitysensor_type)
        sensReading[3] = dist if err == sim.simx_return_ok and dist < self.noDetectionDistance else self.noDetectionDistance
        err, dist = self._readSensor(self.infrared2, sim.sim_object_proximitysensor_type)
        sensReading[4] = dist if err == sim.simx_return_ok and dist < self.noDetectionDistance else self.noDetectionDistance
        return sensReading

    def robotBehaviour(self):
        sensReading = self.readSensors()
        if self.viewSensors:
            print(sensReading)
        
        sensor5, sensor6, ultrasonic, sensor1, sensor2 = sensReading
        threshold = 0.1
        
        if sensor5 < threshold or sensor6 < threshold:
            self.rotate_right()
        elif sensor1 < threshold or sensor2 < threshold:
            self.rotate_left()
        elif ultrasonic < threshold:
            if sensor1 < sensor2:
                self.rotate_left()
            else:
                self.rotate_right()
        else:
            self.move_forward()

class Dr20Robot(Dr20Robot_Interface):
    def __init__(self, sim, clientID):
        Dr20Robot_Interface.__init__(self, sim, clientID)

    def robotBehaviour(self):
        super().robotBehaviour()

def shortPathMain(viewSensors=False, iterations=20):
    print('Program started')
    sim.simxFinish(-1)  # Just in case, close all opened connections
    clientID = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)  # Connect to CoppeliaSim

    if clientID != -1:
        print('Connected to remote API server')
    else:
        print('Connection failed!!')
        sys.exit('Could not connect')

    bot = Dr20Robot(sim, clientID)
    bot.viewSensors = viewSensors

    for _ in range(iterations):
        bot.robotBehaviour()
        time.sleep(0.1)  # Reduced sleep time for more responsive behavior
    
    bot.stop()
    time.sleep(0.5)
    sim.simxFinish(-1)  # Just in case, close all opened connections
    print("...done")

viewSensors = False
try:
    shortPathMain(viewSensors)
except Exception as ex:
    print("Exception was: {}".format(ex))
    sim.simxFinish(-1)  # Just in case, close all opened connections
    print("...uh oh, something went wrong!!!")


Program started
Connected to remote API server
...done
